<a href="https://colab.research.google.com/github/dsa0008/K-Means-Clustering-and-Centroid-Dynamics/blob/main/Data_mining_class_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from sklearn.datasets import fetch_openml

# Fetch the Adult dataset
adult = fetch_openml(name='adult', version=2)

# Data and target
X = adult.data
y = adult.target

# Print some information
print(adult.DESCR)  # Dataset description



/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


**Author**: Ronny Kohavi and Barry Becker  
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Adult) - 1996  
**Please cite**: Ron Kohavi, "Scaling Up the Accuracy of Naive-Bayes Classifiers: a Decision-Tree Hybrid", Proceedings of the Second International Conference on Knowledge Discovery and Data Mining, 1996  

Prediction task is to determine whether a person makes over 50K a year. Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

This is the original version from the UCI repository, with training and test sets merged.

### Variable description

Variables are all self-explanatory except __fnlwgt__. This is a proxy for the demographic background of the people: "People with similar demographic characteristics should have similar weights". This similarity-statement is not transferable across the 51 different states.

Description f

# New Section

In [ ]:
#Convert this in pandas framework to be used for further process
import pandas as pd
# Data and target
X = pd.DataFrame(adult.data, columns=adult.feature_names)
y = pd.Series(adult.target)
# Display the first few rows of the dataset
print(X.head())

# Display the first few rows of the target variable
print(y.head())

# Optionally, if you want to combine the features and target into a single DataFrame:
adult_df = X.copy()
adult_df['income'] = y  # Assuming 'income' is the target variable you're interested in

# Display the first few rows of the combined DataFrame
print(adult_df.head())


    age  workclass    fnlwgt     education  education-num      marital-status  \
0  25.0    Private  226802.0          11th            7.0       Never-married   
1  38.0    Private   89814.0       HS-grad            9.0  Married-civ-spouse   
2  28.0  Local-gov  336951.0    Assoc-acdm           12.0  Married-civ-spouse   
3  44.0    Private  160323.0  Some-college           10.0  Married-civ-spouse   
4  18.0        NaN  103497.0  Some-college           10.0       Never-married   

          occupation relationship   race     sex  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male           0.0           0.0   
1    Farming-fishing      Husband  White    Male           0.0           0.0   
2    Protective-serv      Husband  White    Male           0.0           0.0   
3  Machine-op-inspct      Husband  Black    Male        7688.0           0.0   
4                NaN    Own-child  White  Female           0.0           0.0   

   hours-per-week native-country

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
from scipy.sparse import csr_matrix

# Fetch the Adult dataset
adult = fetch_openml(name='adult', version=2)

# Convert to pandas DataFrame
X = pd.DataFrame(adult.data, columns=adult.feature_names)
y = pd.Series(adult.target).apply(lambda x: ">50K" if x.strip() == ">50K" else "<=50K")

# Replace '?' with numpy.nan and drop rows with any NaN values
X.replace('?', np.nan, inplace=True)
X.dropna(inplace=True)
y = y[X.index].copy()  # Adjust 'y' to match the cleaned 'X'

# Assume all features are categorical for this example
categorical_features = X.columns.tolist()

# Preprocessing pipeline for categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ])

# Function to convert a sparse matrix to a dense matrix
def to_dense(X):
    if isinstance(X, csr_matrix):
        return X.toarray()
    return X

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Naive Bayesian Classifier pipeline
nb_pipeline = make_pipeline(
    preprocessor,
    FunctionTransformer(to_dense, accept_sparse=True),  # Use the to_dense function here
    CategoricalNB()
)

# Fit and evaluate Naive Bayesian Classifier
nb_pipeline.fit(X_train, y_train)
y_pred_nb = nb_pipeline.predict(X_test)

print("Naive Bayesian Classifier Evaluation:")
print(classification_report(y_test, y_pred_nb))

# Decision Tree Classifier pipeline (unchanged)
dt_pipeline = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=42))
dt_pipeline.fit(X_train, y_train)
y_pred_dt = dt_pipeline.predict(X_test)

print("Decision Tree Classifier Evaluation:")
print(classification_report(y_test, y_pred_dt))


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Naive Bayesian Classifier Evaluation:
              precision    recall  f1-score   support

       <=50K       0.91      0.87      0.89      6842
        >50K       0.64      0.74      0.69      2203

    accuracy                           0.84      9045
   macro avg       0.78      0.80      0.79      9045
weighted avg       0.85      0.84      0.84      9045

Decision Tree Classifier Evaluation:
              precision    recall  f1-score   support

       <=50K       0.89      0.92      0.90      6842
        >50K       0.71      0.63      0.67      2203

    accuracy                           0.85      9045
   macro avg       0.80      0.77      0.79      9045
weighted avg       0.84      0.85      0.84      9045



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cleaning the Data

2. Remove records with unknown (?) values from both train and test data sets. For each multi-domain categorical attribute, use one-hot encoding to transform data; for each numerical attribute, use the mean value to transform into binary attribute. (2 points)

  Build k-means clustering algorithm over train data with varied k values (3, 5, 10) based on your chosen distance function and report the centroids of the clusters.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import numpy as np

# Define the column names based on the dataset description
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

# Load the datasets
train_data_path = '/content/drive/MyDrive/Data Mining project1/adult.data' # Update with the actual path
test_data_path = '/content/drive/MyDrive/Data Mining project1/adult.test' # Update with the actual path

train_df = pd.read_csv(train_data_path, names=columns, sep=',\s', na_values="?", engine='python').dropna()
test_df = pd.read_csv(test_data_path, names=columns, sep=',\s', na_values="?", skiprows=1, engine='python').dropna()

# Fix the labels in the test dataset if necessary
train_df['income'] = train_df['income'].str.replace('.', '', regex=False)
test_df['income'] = test_df['income'].str.replace('.', '', regex=False)

# Identify continuous attributes to be converted into binary
continuous_columns = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

# Convert numerical attributes to binary based on their mean
for col in continuous_columns:
    train_mean = train_df[col].mean()
    train_df[col] = (train_df[col] > train_mean).astype(int)

# Prepare data for modeling (exclude target variable and reapply one-hot encoding)
X_train = train_df.drop('income', axis=1)

# Apply one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(X_train)
X_train_encoded = onehot_encoder.transform(X_train)

# Apply k-means clustering with different values of k
k_values = [3, 5, 10]
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_train_encoded)

    # Report the centroids of the clusters
    print(f'Centroids for k={k}:')
    print(kmeans.cluster_centers_)
    print("\n")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Centroids for k=3:
[[ 4.72081218e-01  5.27918782e-01  4.19558686e-02  1.06184606e-01
   6.43944888e-01  5.38692634e-02  8.89878794e-02  6.49539003e-02
   1.03594737e-04  5.79198177e-01  4.20801823e-01 -1.36002321e-15
   1.92901251e-15  1.03594737e-04  3.89445420e-16 -7.35522754e-16
   1.17267307e-15 -7.51135265e-16  9.29244794e-02  1.23381332e-01
   5.21806692e-01  3.87444318e-02  2.44804177e-14  1.66994717e-01
  -7.17741838e-17  5.60447529e-02  2.07334150e-14  1.03594737e-04
   9.99896405e-01  1.25971201e-01  6.21568424e-04  5.47601782e-01
   1.00486895e-02  2.79291412e-01  1.92686212e-02  1.71967264e-02
   8.55692531e-02  2.07189475e-04  6.25712214e-02  2.28840775e-01
   1.63679685e-02  1.02558790e-02  1.59535896e-02  3.28395318e-02
   1.03594737e-03  3.44970475e-01  2.05117580e-02  1.21205843e-01
   4.57888739e-02  1.38816948e-02  4.84616181e-01  2.91619186e-01
   1.30529369e-02  7.50025899e-02  7.74888636e-02  5.82202424e-02
   4.97254739e-03  4.67212266e-02  5.67699161e-02  4.5581

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Centroids for k=5:
[[ 4.11502736e-01  5.88497264e-01  2.78661407e-02  5.78954065e-02
   7.10013997e-01  4.74615091e-02  1.29151292e-01  2.71026848e-02
   5.08970607e-04  5.69156381e-01  4.30843619e-01  3.79183102e-02
   3.74093396e-02  1.23425372e-02  8.39801501e-03  1.62870594e-02
   3.79183102e-02  2.45578318e-02 -1.31838984e-16 -4.25354196e-15
  -4.30211422e-15  6.81746326e-16  5.19531747e-01  4.87110352e-15
   1.78139712e-03 -1.20042865e-15  3.03855452e-01  1.00000000e+00
  -1.29340982e-14  1.03805853e-14  7.63455910e-04  9.98727573e-01
   1.27242652e-04  1.27242652e-04  1.27242652e-04  1.27242652e-04
   4.92429062e-02  2.54485303e-04  2.67591297e-01  1.13373203e-01
   5.75136786e-02  4.89884209e-02  1.00139967e-01  5.48415829e-02
   1.79543880e-16  3.25741188e-02  3.20651482e-02  1.09810408e-01
   2.12495228e-02  1.12355261e-01  9.90584044e-01  3.81727955e-04
   6.61661789e-03  2.41761038e-03 -1.00613962e-14  1.61676228e-15
   1.00521695e-02  1.87046698e-02  6.01857743e-02  7.3800

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Centroids for k=10:
[[ 6.77236045e-15  1.00000000e+00  3.42836521e-02 ...  9.47311440e-01
   1.44352219e-03  3.60880549e-04]
 [ 1.00000000e+00 -6.77236045e-15  1.72413793e-02 ...  9.02093596e-01
   1.53940887e-03  6.15763547e-04]
 [ 1.80072029e-03  9.98199280e-01  4.26170468e-02 ...  8.81752701e-01
   6.00240096e-04  7.04731412e-18]
 ...
 [ 1.19521912e-03  9.98804781e-01  4.06374502e-02 ...  9.25099602e-01
   2.39043825e-03  7.96812749e-04]
 [ 5.77315973e-15  1.00000000e+00  3.72771475e-02 ...  8.95732037e-01
   1.62074554e-03  5.40248514e-04]
 [ 5.76910462e-01  4.23089538e-01  3.64053788e-02 ...  9.20957691e-01
   9.83929157e-04 -8.34835673e-18]]




In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the dataset
adult = fetch_openml(name='adult', version=2, parser='auto')

# Convert to pandas DataFrame and prepare 'X' and 'y'
X = pd.DataFrame(adult.data, columns=adult.feature_names)
y = pd.Series(adult.target).apply(lambda x: ">50K" if x.strip() == ">50K" else "<=50K")

# Assuming the preprocessing (handling missing values, etc.) is already done

# Define the preprocessor
categorical_features = X.columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function to convert a sparse matrix to a dense matrix
def to_dense(X):
    return X.toarray()

# Build the SVM classifier pipeline
svm_pipeline = make_pipeline(
    preprocessor,
    FunctionTransformer(to_dense, accept_sparse=True),  # Convert sparse matrix to dense
    SVC(random_state=42)
)

# Train the SVM classifier on the training data
svm_pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred_svm = svm_pipeline.predict(X_test)

# Evaluate the SVM classifier
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Classifier Evaluation:")
print(f"Accuracy: {accuracy_svm:.4f}")


SVM Classifier Evaluation:
Accuracy: 0.8738


 4. Continue using the train datasets from step 2, build a neural network classifier and report the predicted accuracy of the test data.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the dataset
adult = fetch_openml(name='adult', version=2, parser='auto')

# Convert to pandas DataFrame and prepare 'X' and 'y'
X = pd.DataFrame(adult.data, columns=adult.feature_names)
y = pd.Series(adult.target).apply(lambda x: ">50K" if x.strip() == ">50K" else "<=50K")

# Assuming the preprocessing (handling missing values, etc.) is already done

# Define the preprocessor
categorical_features = X.columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the MLPClassifier: Using a single hidden layer with 50 neurons as a starting point
nn_classifier = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)

# Apply one-hot encoding through the preprocessor to avoid future errors
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

# Train the neural network classifier on the training data
nn_classifier.fit(X_train_encoded, y_train)

# Predict the labels of the test set
y_pred_nn = nn_classifier.predict(X_test_encoded)

# Calculate the accuracy of the predictions
accuracy_nn = accuracy_score(y_test, y_pred_nn)

# Print the accuracy
print(f'Neural Network Classifier Accuracy on Test Data: {accuracy_nn * 100:.2f}%')


Neural Network Classifier Accuracy on Test Data: 85.04%
